# Using EcoFOCIpy to process raw field data

## DY1806 (Spring Mooring Cruise - Dyson)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [2]:
import yaml
import glob

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [3]:
sample_data_dir = '/Users/bell/ecoraid/2018/CTDcasts/dy1807l3/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [12]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'DY1807L3' #no hyphens
cruise_meta_file = sample_data_dir+'logs/DY1807L3.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTD.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2018/CTDcasts/dy1807l3/rawconverted/ctd036.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1807l3/rawconverted/ctd039.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1807l3/rawconverted/ctd042.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1807l3/rawconverted/ctd045.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1807l3/rawconverted/ctd048.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1807l3/rawconverted/ctd050.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1807l3/rawconverted/ctd051.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1807l3/rawconverted/ctd052.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1807l3/rawconverted/ctd053.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1807l3/rawconverted/ctd054.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1807l3/rawconverted/ctd055.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1807l3/rawconverted/ctd056.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1807l3/rawconvert

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

In [13]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['ctd050.btl'].describe()

,sbeox0ml/l,sbeox0ps,sbox0mm/kg,sbeox1ml/l,sbeox1ps,sbox1mm/kg,sal00,sal11,sigma-t00,sigma-t11,fleco-afl,t090c,t190c,turbwetntu0,prdm,scan,datetime
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.00000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10
mean,6.405100,92.774800,278.878500,-1.530370,-22.046300,-66.630300,32.838270,32.838240,25.72818,25.728240,8.715050,6.586820,6.586140,0.319220,52.011700,19198.200000,2018-08-16 10:41:08.500000256
min,5.643900,76.097000,245.616000,-1.659300,-22.372000,-72.212000,32.742400,32.743300,25.17340,25.173300,-0.320600,3.597900,3.598300,0.113800,2.614000,8789.000000,2018-08-16 10:33:55
25%,6.386475,89.778500,278.202250,-1.627725,-22.193000,-70.845500,32.745325,32.745125,25.22605,25.226125,1.978700,4.079700,4.079825,0.195775,22.840750,15892.500000,2018-08-16 10:38:51
50%,6.439400,96.335500,280.510500,-1.568750,-22.053000,-68.291500,32.840350,32.840450,25.91880,25.919150,10.106650,5.374650,5.372450,0.372550,45.598500,20072.000000,2018-08-16 10:41:45
75%,6.649075,100.166000,289.404250,-1.406475,-21.852750,-61.268000,32.891500,32.892100,26.10085,26.101275,12.904300,9.787525,9.787250,0.420575,72.165000,23138.750000,2018-08-16 10:43:52.500000
max,7.085200,101.016000,308.456000,-1.392600,-21.791000,-60.668000,32.976300,32.975500,26.21510,26.214400,18.847300,10.111700,10.111500,0.461200,126.243000,26173.000000,2018-08-16 10:45:59
std,0.451364,9.657849,19.676072,0.114752,0.210856,4.967006,0.093360,0.093131,0.45727,0.457119,7.082316,2.909368,2.909160,0.133980,40.049689,5528.010046,NaN


## Create BTL report file

In [14]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd001':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [15]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'DY1807L3',
 'CruiseID_Historic': '',
 'CruiseID_Alternates': 'dy1807L3a,dy1807L3b',
 'Project_Leg': 'L3',
 'Vessel': 'NOAAS Oscar Dyson',
 'ShipID': 'DY',
 'StartDate': datetime.date(2018, 7, 24),
 'EndDate': datetime.date(2018, 8, 26),
 'Project': 'MACE',
 'ChiefScientist': '',
 'StartPort': 'Dutch Harbor, AK',
 'EndPort': 'Dutch Harbor, AK',
 'CruiseLocation': 'Bering Sea',
 'Description': 'Bering Sea Summer Pollack',
 'CruiseYear': 2018,
 'ctdlogs_pdf_name': 'DY1807L3_CastLogs.pdf',
 'Accession': '',
 'Links': '\r',
 'QC_Status': '',
 'QC_Signature': ''}

In [16]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD050']

{'id': 22962,
 'Vessel': 'R/V Oscar Dyson',
 'CruiseID': 'DY1807',
 'Project_Leg': 'L3',
 'UniqueCruiseID': 'DY1807L3',
 'Project': 'Bering Sea Summer Pollack',
 'StationNo_altname': '50',
 'ConsecutiveCastNo': 'CTD050',
 'LatitudeDeg': 59,
 'LatitudeMin': 0.15,
 'LongitudeDeg': 174,
 'LongitudeMin': 59.91,
 'GMTDay': 16,
 'GMTMonth': 'Aug',
 'GMTYear': 2018,
 'GMTTime': 38040,
 'DryBulb': 8.6,
 'RelativeHumidity': 84,
 'WetBulb': -99.9,
 'Pressure': 1018,
 'SeaState': '-99',
 'Visibility': '-99',
 'WindDir': 140,
 'WindSpd': 2.0,
 'CloudAmt': '-99',
 'CloudType': '-99',
 'Weather': '-99',
 'SurfaceTemp': -99.9,
 'BottomDepth': 130,
 'StationNameID': '',
 'MaxDepth': 125,
 'InstrumentSerialNos': 'Press SN =291, Pri Temp SN = 0188, Sec Temp SN =0241, Pri Cond SN = 04-2985, Sec Cond Sn =04-3127',
 'Notes': 'pycnocline @ 35m',
 'NutrientBtlNiskinNo': '1;2;3;4;5;6;7;8;9;10;11',
 'NutrientBtlNumbers': '391;;392;393;394;395;396;3973;98;399;400',
 'OxygenBtlNiskinNo': '',
 'OxygenBtlNumbers':

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [17]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [20]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['ctd050.btl'] = cruise_data['ctd050.btl'].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            'sbox1mm/kg':'oxy_conc_ch2',
                            'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

cruise_data['ctd050.btl'].sample()

,oxy_concM_ch1,oxy_percentsat_ch1,oxy_conc_ch1,oxy_concM_ch2,oxy_percentsat_ch2,oxy_conc_ch2,salinity_ch1,salinity_ch2,sigma_t_ch1,sigma_t_ch2,chlor_fluorescence,temperature_ch1,temperature_ch2,turbidity,pressure,scan,datetime,cast
bottle,,,,,,,,,,,,,,,,,,
6.0,6.7612,93.733,294.302,-1.5937,-22.095,-69.372,32.8572,32.8569,26.0009,26.0007,8.89,4.7791,4.779,0.1603,50.814,19272.0,2018-08-16 10:41:12,ctd050


In [22]:
cruise_data['ctd050.btl'].columns

Index(['oxy_concM_ch1', 'oxy_percentsat_ch1', 'oxy_conc_ch1', 'oxy_concM_ch2',
       'oxy_percentsat_ch2', 'oxy_conc_ch2', 'salinity_ch1', 'salinity_ch2',
       'sigma_t_ch1', 'sigma_t_ch2', 'chlor_fluorescence', 'temperature_ch1',
       'temperature_ch2', 'turbidity', 'pressure', 'scan', 'datetime', 'cast'],
      dtype='object')

## Add institutional meta-information


In [23]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [26]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['ctd050.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [27]:
cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 2kB
Dimensions:             (bottle: 10)
Coordinates:
  * bottle              (bottle) float64 80B 1.0 3.0 4.0 5.0 ... 9.0 10.0 11.0
Data variables: (12/18)
    oxy_concM_ch1       (bottle) float64 80B 5.644 5.654 6.552 ... 6.432 6.418
    oxy_percentsat_ch1  (bottle) float64 80B 76.1 76.25 89.05 ... 100.1 100.4
    oxy_conc_ch1        (bottle) float64 80B 245.6 246.1 285.2 ... 280.2 279.6
    oxy_concM_ch2       (bottle) float64 80B -1.659 -1.654 ... -1.403 -1.393
    oxy_percentsat_ch2  (bottle) float64 80B -22.37 -22.31 ... -21.82 -21.79
    oxy_conc_ch2        (bottle) float64 80B -72.21 -71.99 ... -61.09 -60.67
    ...                  ...
    temperature_ch2     (bottle) float64 80B 3.598 3.603 3.947 ... 9.844 10.11
    turbidity           (bottle) float64 80B 0.3615 0.3022 ... 0.3836 0.3896
    pressure            (bottle) float64 80B 126.2 102.0 75.91 ... 10.3 2.614
    scan                (bottle) float64 80B 8.789e+03 1.316e+04 ... 2.617e+04
    datetime            (bottle) datetime64[ns] 80B 2018-08-16T10:33:55 ... 2...
    cast                (bottle) object 80B 'ctd050' 'ctd050' ... 'ctd050'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [28]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [30]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['ctd050.btl'].columns.values),drop_missing=True)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [31]:
cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 1kB
Dimensions:             (latitude: 1, longitude: 1, time: 1, bottle: 10)
Coordinates:
  * bottle              (bottle) float64 80B 1.0 3.0 4.0 5.0 ... 9.0 10.0 11.0
  * latitude            (latitude) float64 8B 1e+35
  * longitude           (longitude) float64 8B 1e+35
  * time                (time) float64 8B 1e+35
Data variables: (12/15)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 80B 5.644...
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 80B 76.1 ...
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 80B 245.6...
    oxy_concM_ch2       (latitude, longitude, time, bottle) float64 80B -1.65...
    oxy_percentsat_ch2  (latitude, longitude, time, bottle) float64 80B -22.3...
    oxy_conc_ch2        (latitude, longitude, time, bottle) float64 80B -72.2...
    ...                  ...
    sigma_t_ch2         (latitude, longitude, time, bottle) float64 80B 26.21...
    chlor_fluorescence  (latitude, longitude, time, bottle) float64 80B -0.18...
    temperature_ch1     (latitude, longitude, time, bottle) float64 80B 3.598...
    temperature_ch2     (latitude, longitude, time, bottle) float64 80B 3.598...
    turbidity           (latitude, longitude, time, bottle) float64 80B 0.361...
    pressure            (latitude, longitude, time, bottle) float64 80B 126.2...

In [33]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD050')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 1kB
Dimensions:             (latitude: 1, longitude: 1, time: 1, bottle: 10)
Coordinates:
  * bottle              (bottle) float64 80B 1.0 3.0 4.0 5.0 ... 9.0 10.0 11.0
  * latitude            (latitude) float64 8B 1e+35
  * longitude           (longitude) float64 8B 1e+35
  * time                (time) float64 8B 1e+35
Data variables: (12/15)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 80B 5.644...
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 80B 76.1 ...
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 80B 245.6...
    oxy_concM_ch2       (latitude, longitude, time, bottle) float64 80B -1.65...
    oxy_percentsat_ch2  (latitude, longitude, time, bottle) float64 80B -22.3...
    oxy_conc_ch2        (latitude, longitude, time, bottle) float64 80B -72.2...
    ...                  ...
    sigma_t_ch2         (latitude, longitude, time, bottle) float64 80B 26.21...
    chlor_fluorescence  (latitude, longitude, time, bottle) float64 80B -0.18...
    temperature_ch1     (latitude, longitude, time, bottle) float64 80B 3.598...
    temperature_ch2     (latitude, longitude, time, bottle) float64 80B 3.598...
    turbidity           (latitude, longitude, time, bottle) float64 80B 0.361...
    pressure            (latitude, longitude, time, bottle) float64 80B 126.2...
Attributes: (12/48)
    CruiseID:                      DY1807L3
    CruiseID_Historic:             
    CruiseID_Alternates:           dy1807L3a,dy1807L3b
    Project_Leg:                   L3
    Vessel:                        NOAAS Oscar Dyson
    ShipID:                        DY
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2025-12-03T08:26:51Z
    date_modified:                 
    QC_indicator:                  unqcd

## Rare Bottle File Edits

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [34]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            # 'sbox1mm/kg':'oxy_conc_ch2',
                            # 'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            # 'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=True)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history